In [5]:
# Import libraries
import pandas as pd
import pickle
with open("models/classical_model.pkl", "rb") as f:
    saved_data = pickle.load(f)

model = saved_data["model"]   
vectorizer = saved_data["vectorizer"] 
le = saved_data["label_encoder"]    

# Function to predict sentiment
def predict_sentiment(text):
    text_tfidf = vectorizer.transform([text])
    pred_encoded = model.predict(text_tfidf)[0]
    pred_label = le.inverse_transform([pred_encoded])[0]
    return pred_label

tweets = ["Not sure how I feel about the recent updates…","The service was terrible, very disappointed."
]
results = []
for tweet in tweets:
    sentiment = predict_sentiment(tweet)
    results.append({"Tweet": tweet, "Predicted Sentiment": sentiment})

df_results = pd.DataFrame(results)
print(df_results)


                                           Tweet Predicted Sentiment
0  Not sure how I feel about the recent updates…             neutral
1   The service was terrible, very disappointed.                 bad


In [1]:
import streamlit as st
import pickle
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import numpy as np

# Load classical model
with open("models/classical_model.pkl", "rb") as f:
    data = pickle.load(f)
    clf = data["model"]
    tfidf = data["vectorizer"]
    le = data["label_encoder"]

# Load DistilBERT model and tokenizer
bert_model = DistilBertForSequenceClassification.from_pretrained("models/transformer_model")
tokenizer = DistilBertTokenizerFast.from_pretrained("models/transformer_model")

# Streamlit app
st.title("Sentiment Analysis App")
st.write("Analyze your text using Logistic Regression (TF-IDF) or DistilBERT.")

# Text input
user_input = st.text_area("Enter text here:", "")

# Model selection
model_choice = st.selectbox("Choose model:", ["Logistic Regression", "DistilBERT"])

if st.button("Predict"):
    if not user_input.strip():
        st.warning("Please enter some text!")
    else:
        if model_choice == "Logistic Regression":
            # Classical model prediction
            X_input = tfidf.transform([user_input])
            pred = clf.predict(X_input)[0]
            label = le.inverse_transform([pred])[0]
            st.success(f"Predicted Sentiment: {label}")

        elif model_choice == "DistilBERT":
            # DistilBERT prediction
            inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=64)
            outputs = bert_model(**inputs)
            pred = torch.argmax(outputs.logits, dim=1).item()
            label = le.inverse_transform([pred])[0]
            st.success(f"Predicted Sentiment: {label}")


2025-12-17 16:09:37.812 
  command:

    streamlit run C:\Users\techmania\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
